1.) Set Up Environment

In [1]:
import pandas as pd #dataframe library
import numpy as np #numeric library
import datetime as dt

#plot library
import matplotlib.pyplot as plt 
import seaborn as sns 

#evaluation library
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score

#label encoder library
from sklearn.preprocessing import LabelEncoder

# for min_max scaling (Kaggle)
from mlxtend.preprocessing import minmax_scaling

from sklearn.preprocessing import MinMaxScaler, StandardScaler

# for changing datatime type
from sklearn.preprocessing import StandardScaler

# for Box-Cox Transformation
from scipy import stats

# set seed for reproducibility
np.random.seed(0)

2.) Dataset Preparation

In [2]:
# for SMP Team
smp_url = 'https://docs.google.com/spreadsheets/d/1fQjK_yvaj5yh006nRTcMXaoU0RAx8OUy2IhpZG4tNMw/edit#gid=0'
smp_data = smp_url.replace('/edit#gid=', '/export?format=csv&gid=')
smp = pd.read_csv(smp_data)
smp.head()

,subs_no
0,2106705274
1,1075588229
2,1349135784
3,68147238
4,1356074904


In [3]:
# for NS Team
ns_url = 'https://docs.google.com/spreadsheets/d/16Vrap9JRBTrq9vA-Xz-1ZkLv0GiaWZ6oETrhL4ry-Ok/edit#gid=0'
ns_data = ns_url.replace('/edit#gid=', '/export?format=csv&gid=')
ns = pd.read_csv(ns_data)
ns.head()

,time_no,subs_key,subs_no,service_filter,rem_bal,rtd_amt,free_unit,usg_unit,sid
0,1/9/2021 21:24,6281932975189,2106705274,SMSBAS,673,50,0.0,1,SILVER
1,1/9/2021 21:29,6281932975189,2106705274,SMSBAS,623,50,0.0,1,SILVER
2,1/9/2021 21:36,6281932975189,2106705274,SMSBAS,573,50,0.0,1,SILVER
3,1/9/2021 7:43,6287775860056,1075588229,VASOCC,7220,5500,0.0,1,SILVER
4,1/9/2021 4:24,6287775860056,1075588229,CONTEN,101720,55,NaN,1048576,TRIBE


In [4]:
# for ITInfra Team
infra_url = 'https://docs.google.com/spreadsheets/d/1RxhtYZPFhB5F2a3434SAC6K11j_sRMhRDu5avKHMAQg/edit#gid=0'
infra_data = infra_url.replace('/edit#gid=', '/export?format=csv&gid=')
infra = pd.read_csv(infra_data)
infra.head()

,day_no,time_no,service_filter,subs_no,b_no,event_action_code,prefix_dom_sk_id,originating_cell_id,home_poc,city_name,physical_poc,roam_zone,destination_poc
0,1/9/2021,1/9/2021 21:24,SMSBAS,2106705274,6.285693e+12,O,5636,510.11.34314.18692,JK0,JAKARTA,KA0,LOCXLM,JKT
1,1/9/2021,1/9/2021 21:29,SMSBAS,2106705274,6.285693e+12,O,5636,510.11.34314.38401,JK0,JAKARTA,KA0,LOCXLM,JKT
2,1/9/2021,1/9/2021 21:36,SMSBAS,2106705274,6.285693e+12,O,5636,510.11.34314.18693,JK0,JAKARTA,KA0,LOCXLM,JKT
3,1/9/2021,1/9/2021 7:43,VASOCC,1075588229,9.955702e+06,B,-1,0.0.0.0,NOPOC,JAKARTA,B,LOCXLM,0
4,1/9/2021,1/9/2021 4:24,CONTEN,1075588229,0.000000e+00,O,-1,510.11.411042.3,JK0,JAKARTA,JK0,NATZ10,0


In [5]:
# for Finance Team
fin_url = 'https://docs.google.com/spreadsheets/d/1F0KiwUmqIiknViTQ4Os_SdHdBVA2ErOOrSgH2Hqeumc/edit#gid=0'
fin_data = fin_url.replace('/edit#gid=', '/export?format=csv&gid=')
fin = pd.read_csv(fin_data)
fin.head()

,subs_no,payment_cat
0,2106705274,PRE
1,1075588229,PRE
2,1349135784,PRE
3,68147238,PRE
4,1356074904,PRE


3.) Merge Columns

In [6]:
# Create a new column in infra to store the merged data
infra['payment_cat'] = ''


# Iterate over the data in infra
for index, row in infra.iterrows():
    subsno_infra = row['subs_no']
    
    # Find matching rows in fin based on the conditions
    match = fin[
        (fin['subs_no'] == subsno_infra) 
    ]
    
    # Copy the desired data from network_data to the new columns in infra_data
    if not match.empty:
        infra.at[index, 'payment_cat'] = match.iloc[0]['payment_cat']

In [7]:
infra.head()

,day_no,time_no,service_filter,subs_no,b_no,event_action_code,prefix_dom_sk_id,originating_cell_id,home_poc,city_name,physical_poc,roam_zone,destination_poc,payment_cat
0,1/9/2021,1/9/2021 21:24,SMSBAS,2106705274,6.285693e+12,O,5636,510.11.34314.18692,JK0,JAKARTA,KA0,LOCXLM,JKT,PRE
1,1/9/2021,1/9/2021 21:29,SMSBAS,2106705274,6.285693e+12,O,5636,510.11.34314.38401,JK0,JAKARTA,KA0,LOCXLM,JKT,PRE
2,1/9/2021,1/9/2021 21:36,SMSBAS,2106705274,6.285693e+12,O,5636,510.11.34314.18693,JK0,JAKARTA,KA0,LOCXLM,JKT,PRE
3,1/9/2021,1/9/2021 7:43,VASOCC,1075588229,9.955702e+06,B,-1,0.0.0.0,NOPOC,JAKARTA,B,LOCXLM,0,PRE
4,1/9/2021,1/9/2021 4:24,CONTEN,1075588229,0.000000e+00,O,-1,510.11.411042.3,JK0,JAKARTA,JK0,NATZ10,0,PRE


In [8]:
total_rows= len(infra)
print(total_rows)

43947


In [9]:
# Create a new column in infra to store the merged data
infra['subs_key'] = ''
infra['rem_bal'] = ''
infra['rtd_amt'] = ''
infra['free_unit'] = ''
infra['usg_unit'] = ''
infra['sid'] = ''

# Iterate over the data in infra
for index, row in infra.iterrows():
    servicefilter_infra = row['service_filter']
    subsno_infra = row['subs_no']
    timeno_infra = row['time_no']
    
    # Find matching rows in network_data based on the conditions
    match = ns[
        (ns['service_filter'] == servicefilter_infra) &
        (ns['subs_no'] == subsno_infra) &
        (ns['time_no'] == timeno_infra)
    ]
    
    # Copy the desired data from network_data to the new columns in infra_data
    if not match.empty:
        infra.at[index, 'subs_key'] = match.iloc[0]['subs_key']
        infra.at[index, 'rem_bal'] = match.iloc[0]['rem_bal']
        infra.at[index, 'rtd_amt'] = match.iloc[0]['rtd_amt']
        infra.at[index, 'free_unit'] = match.iloc[0]['free_unit']
        infra.at[index, 'usg_unit'] = match.iloc[0]['usg_unit']
        infra.at[index, 'sid'] = match.iloc[0]['sid']

In [10]:
infra.head()

,day_no,time_no,service_filter,subs_no,b_no,event_action_code,prefix_dom_sk_id,originating_cell_id,home_poc,city_name,physical_poc,roam_zone,destination_poc,payment_cat,subs_key,rem_bal,rtd_amt,free_unit,usg_unit,sid
0,1/9/2021,1/9/2021 21:24,SMSBAS,2106705274,6.285693e+12,O,5636,510.11.34314.18692,JK0,JAKARTA,KA0,LOCXLM,JKT,PRE,6281932975189,673,50,0.0,1,SILVER
1,1/9/2021,1/9/2021 21:29,SMSBAS,2106705274,6.285693e+12,O,5636,510.11.34314.38401,JK0,JAKARTA,KA0,LOCXLM,JKT,PRE,6281932975189,623,50,0.0,1,SILVER
2,1/9/2021,1/9/2021 21:36,SMSBAS,2106705274,6.285693e+12,O,5636,510.11.34314.18693,JK0,JAKARTA,KA0,LOCXLM,JKT,PRE,6281932975189,573,50,0.0,1,SILVER
3,1/9/2021,1/9/2021 7:43,VASOCC,1075588229,9.955702e+06,B,-1,0.0.0.0,NOPOC,JAKARTA,B,LOCXLM,0,PRE,6287775860056,7220,5500,0.0,1,SILVER
4,1/9/2021,1/9/2021 4:24,CONTEN,1075588229,0.000000e+00,O,-1,510.11.411042.3,JK0,JAKARTA,JK0,NATZ10,0,PRE,6287775860056,101720,55,NaN,1048576,TRIBE


In [11]:
total_rows= len(infra)
print(total_rows)

43947


4.) Make a Copy 

In [12]:
data = infra.copy()

5.) Change Name and Order of Columns)

In [13]:
print(data.columns)

Index(['day_no', 'time_no', 'service_filter', 'subs_no', 'b_no',
       'event_action_code', 'prefix_dom_sk_id', 'originating_cell_id',
       'home_poc', 'city_name', 'physical_poc', 'roam_zone', 'destination_poc',
       'payment_cat', 'subs_key', 'rem_bal', 'rtd_amt', 'free_unit',
       'usg_unit', 'sid'],
      dtype='object')


In [14]:
col_name = [
    'date', #date
    'date_time' , #date and time
    'service_type', #service_type
    'subs_no', #subs_no
    'destination_no', #b_no
    'event_action_code', #event_action_code
    'prefix_domain', #prefix_dom_sk_id
    'cell_id', #originating_cell_id
    'home_poc', #home_poc
    'city_name', #city_name
    'physical_poc', #physical_poc
    'roam_zone', #roam_zone
    'destination_poc', #destination_poc
    'payment_type', #payment_cat
    'subs_id', #subs_key
    'remaining_balance', #rem_bal
    'amount_of_trx', #rtd_amt
    'free_unit', #free_unit
    'usage_unit', #usg_unit
    'sid' #sid
]

data.columns = col_name
print(data.columns)

Index(['date', 'date_time', 'service_type', 'subs_no', 'destination_no',
       'event_action_code', 'prefix_domain', 'cell_id', 'home_poc',
       'city_name', 'physical_poc', 'roam_zone', 'destination_poc',
       'payment_type', 'subs_id', 'remaining_balance', 'amount_of_trx',
       'free_unit', 'usage_unit', 'sid'],
      dtype='object')


In [15]:
order = [
    'date', #date
    'date_time' , #date and time
    'subs_no', #subs_no
    'subs_id', #subs_key
    'payment_type', #payment_cat
    'service_type', #service_type
    'destination_no', #b_no
    'event_action_code', #event_action_code
    'prefix_domain', #prefix_dom_sk_id
    'cell_id', #originating_cell_id
    'home_poc', #home_poc
    'city_name', #city_name
    'physical_poc', #physical_poc
    'roam_zone', #roam_zone
    'destination_poc', #destination_poc
    'remaining_balance', #rem_bal
    'amount_of_trx', #rtd_amt
    'free_unit', #free_unit
    'usage_unit', #usg_unit
    'sid' #sid
]

data = data[order]


In [16]:
data.head()

,date,date_time,subs_no,subs_id,payment_type,service_type,destination_no,event_action_code,prefix_domain,cell_id,home_poc,city_name,physical_poc,roam_zone,destination_poc,remaining_balance,amount_of_trx,free_unit,usage_unit,sid
0,1/9/2021,1/9/2021 21:24,2106705274,6281932975189,PRE,SMSBAS,6.285693e+12,O,5636,510.11.34314.18692,JK0,JAKARTA,KA0,LOCXLM,JKT,673,50,0.0,1,SILVER
1,1/9/2021,1/9/2021 21:29,2106705274,6281932975189,PRE,SMSBAS,6.285693e+12,O,5636,510.11.34314.38401,JK0,JAKARTA,KA0,LOCXLM,JKT,623,50,0.0,1,SILVER
2,1/9/2021,1/9/2021 21:36,2106705274,6281932975189,PRE,SMSBAS,6.285693e+12,O,5636,510.11.34314.18693,JK0,JAKARTA,KA0,LOCXLM,JKT,573,50,0.0,1,SILVER
3,1/9/2021,1/9/2021 7:43,1075588229,6287775860056,PRE,VASOCC,9.955702e+06,B,-1,0.0.0.0,NOPOC,JAKARTA,B,LOCXLM,0,7220,5500,0.0,1,SILVER
4,1/9/2021,1/9/2021 4:24,1075588229,6287775860056,PRE,CONTEN,0.000000e+00,O,-1,510.11.411042.3,JK0,JAKARTA,JK0,NATZ10,0,101720,55,NaN,1048576,TRIBE


6.) Dropping Un-Needed Columns

In [17]:
data = data.drop(['subs_id','destination_no','cell_id','prefix_domain','event_action_code', 'roam_zone', 'sid'], axis = 1)

In [18]:
data.head()

,date,date_time,subs_no,payment_type,service_type,home_poc,city_name,physical_poc,destination_poc,remaining_balance,amount_of_trx,free_unit,usage_unit
0,1/9/2021,1/9/2021 21:24,2106705274,PRE,SMSBAS,JK0,JAKARTA,KA0,JKT,673,50,0.0,1
1,1/9/2021,1/9/2021 21:29,2106705274,PRE,SMSBAS,JK0,JAKARTA,KA0,JKT,623,50,0.0,1
2,1/9/2021,1/9/2021 21:36,2106705274,PRE,SMSBAS,JK0,JAKARTA,KA0,JKT,573,50,0.0,1
3,1/9/2021,1/9/2021 7:43,1075588229,PRE,VASOCC,NOPOC,JAKARTA,B,0,7220,5500,0.0,1
4,1/9/2021,1/9/2021 4:24,1075588229,PRE,CONTEN,JK0,JAKARTA,JK0,0,101720,55,NaN,1048576


7.) Missing Values

In [19]:
missing_values_count = data.isnull().sum() + data.isin(['', ' ','00000000', None, np.nan]).sum()
missing_values_count

date                     0
date_time                0
subs_no                  0
payment_type             0
service_type             0
home_poc                 0
city_name                0
physical_poc             0
destination_poc      20229
remaining_balance        0
amount_of_trx            0
free_unit            14646
usage_unit               0
dtype: int64

In [ ]:
#for free_unit
#jika free_unit memiliki nilai NULL, maka service_filter pada baris dengan nilai free_unit == NULL tersebut tidak memiliki fitur free_unit, sehingga nilai NULL harus diganti dengan nilai 0.

In [21]:
data['free_unit'] = data['free_unit'].fillna(0)
data['free_unit'] = data['free_unit'].replace(['', ' ', None], 0)

In [27]:
# for destination_poc
#00000000 --> service filter: CONTEN/VASOCC
#0 --> service filter:CONTEN/VASOCC
#(Blank) --> service filter: INTNAL/NATNAL
# so, we will fill the missing values of destination_poc with NOPOC

In [25]:
data['destination_poc'] = data['destination_poc'].fillna('NOPOC')
data['destination_poc'] = data['destination_poc'].replace(['', ' ', None, '00000000'], 'NOPOC')


In [26]:
missing_values_count = data.isnull().sum() + data.isin(['', ' ','00000000', None, np.nan]).sum()
missing_values_count

date                 0
date_time            0
subs_no              0
payment_type         0
service_type         0
home_poc             0
city_name            0
physical_poc         0
destination_poc      0
remaining_balance    0
amount_of_trx        0
free_unit            0
usage_unit           0
dtype: int64

8.) Data Type Conversion for Numerical Data 